# PROJET ANALYSE FINANCIERE

scrapper des données financières sur internet , ici via selenium

    nb : charger le geodriver.exe au même endroit que le notebook (cf installation selenium)
    détail des opé dans apple

SOURCES :
https://selenium-python.readthedocs.io/installation.html

https://www.selenium.dev/documentation/fr/webdriver/locating_elements/

https://www.ionos.fr/digitalguide/sites-internet/developpement-web/tutoriel-selenium-webdriver/

https://pythonexamples.org/python-re-findall/

site de référence : https://www.nasdaq.com/market-activity/stocks/aapl/historical

In [7]:
#Importing packages
from selenium import webdriver
import pandas as pd
from time import sleep
driver = webdriver.Firefox(executable_path = "geckodriver.exe")

In [8]:
# On charge la page
driver.get('https://www.nasdaq.com/market-activity/stocks/aapl/historical/')

# On crée un dataframe
apple = pd.DataFrame(columns = ['Date','Close/Last','Volume', 'Open', 'Hight', 'Low'])

# On laisse la page charger
sleep(5)

# On clique sur le bouton 1y
bouton_1y = driver.find_element_by_xpath('/html/body/div[2]/div/main/div[2]/div[4]/div[3]/div/div[1]/div/div[1]/div[3]/div/div/div/button[4]')
bouton_1y.click()
sleep(2)

# Tant que le bouton n'est pas déactivé 
is_actived = True

while is_actived:

    nb_tr = driver.find_element_by_xpath('/html/body/div[2]/div/main/div[2]/div[4]/div[3]/div/div[1]/div/div[1]/div[4]/div[2]/table/tbody[2]')
    nb_ligne = nb_tr.text.split('\n')
    row = []
    for elt in nb_ligne:
        elt = elt.split(' ')
        for i in range(len(elt)):
            # C'est l'index 2 soit la colonne volume
            if i == 2:
                elt[i] = int(elt[i].replace(',',''))
            # Sinon si l'index n'est pas une date
            elif i != 0:
                elt[i] = float(elt[i].replace("$",'').replace(',',''))
        apple.loc[len(apple)] = elt
    
    next_page = driver.find_element_by_xpath("/html/body/div[2]/div/main/div[2]/div[4]/div[3]/div/div[1]/div/div[1]/div[5]/button[2]")

    # La variable is_disabled prend la valeur de l'attribut "disabled" et donc ici soit "true" ou "false"
    is_disabled = next_page.get_attribute("disabled")

    # Si la variable is_disabled est égale à true, cela veut dire que nous sommes arrivés à la fin du défilement
    # et donc on doit arrêter notre boucle en passant la variable is_actived à false
    if is_disabled == "true":
        is_actived = False
    # Sinon, on clique sur l'élément pour aller à la page suivante et continuer le scrapping.
    else:
        next_page.click()
    sleep(6)
# J'affiche le dataframe
apple

ElementClickInterceptedException: Message: Element <button class="pagination__next"> is not clickable at point (805,384) because another element <div id="_evidon_banner" class="evidon-banner"> obscures it


In [3]:
#transformation du fichier panda en csv
apple.to_csv('apple.csv', index=False)

In [4]:
#autre façon de transformer un fichier en csv , non utilisée ici (formule Amaury)

def append_list_as_row(nom_csv, liste):
    # On ouvre le csv en mode append :
    with open(nom_csv, 'a+', newline='') as csvfile:
        # Create a writer object from csv module
        csv_writer = writer(csvfile)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(liste)

In [3]:
data=pd.read_csv("apple.csv")

In [4]:
data

,Date,Close/Last,Volume,Open,Hight,Low
0,03/17/2021,124.7600,111932600,124.0500,125.8599,122.3360
1,03/16/2021,125.5700,115227900,125.7000,127.2200,124.7150
2,03/15/2021,123.9900,92590560,121.4100,124.0000,120.4200
3,03/12/2021,121.0300,88105050,120.4000,121.1700,119.1600
4,03/11/2021,121.9600,103026500,122.5400,123.2100,121.2600
...,...,...,...,...,...,...
247,03/24/2020,61.7200,287531080,59.0900,61.9225,58.5750
248,03/23/2020,56.0925,336752840,57.0200,57.1249,53.1525
249,03/20/2020,57.3100,401693200,61.7950,62.9575,57.0000
250,03/19/2020,61.1950,271857040,61.8462,63.2100,60.6525


In [5]:
data_fini = data
data_fini = data_fini.rename(columns={'Close/Last':'Close'})
data_fini['Date'] = pd.to_datetime(data_fini['Date'], dayfirst = True)
data_fini

,Date,Close,Volume,Open,Hight,Low
0,2021-03-17,124.7600,111932600,124.0500,125.8599,122.3360
1,2021-03-16,125.5700,115227900,125.7000,127.2200,124.7150
2,2021-03-15,123.9900,92590560,121.4100,124.0000,120.4200
3,2021-12-03,121.0300,88105050,120.4000,121.1700,119.1600
4,2021-11-03,121.9600,103026500,122.5400,123.2100,121.2600
...,...,...,...,...,...,...
247,2020-03-24,61.7200,287531080,59.0900,61.9225,58.5750
248,2020-03-23,56.0925,336752840,57.0200,57.1249,53.1525
249,2020-03-20,57.3100,401693200,61.7950,62.9575,57.0000
250,2020-03-19,61.1950,271857040,61.8462,63.2100,60.6525


In [38]:
data_fini.to_csv('apple_fini.csv', index=False)

In [17]:
for index in data.index:
    day = str(data["Date"][index])[:2]
    month = str(data["Date"][index])[3:]
    french_date = day + "-" + month
    english_date = "2021-" + month + "-" + day
    data["Date"][index] = data["Date"][index].replace(french_date, english_date)

<ipython-input-17-aad2b174d44d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Date"][index] = data["Date"][index].replace(french_date, english_date)


In [15]:
data

,Date,Close/Last,Volume,Open,Hight,Low
0,03/17/2021,124.7600,111932600,124.0500,125.8599,122.3360
1,03/16/2021,125.5700,115227900,125.7000,127.2200,124.7150
2,03/15/2021,123.9900,92590560,121.4100,124.0000,120.4200
3,03/12/2021,121.0300,88105050,120.4000,121.1700,119.1600
4,03/11/2021,121.9600,103026500,122.5400,123.2100,121.2600
...,...,...,...,...,...,...
247,03/24/2020,61.7200,287531080,59.0900,61.9225,58.5750
248,03/23/2020,56.0925,336752840,57.0200,57.1249,53.1525
249,03/20/2020,57.3100,401693200,61.7950,62.9575,57.0000
250,03/19/2020,61.1950,271857040,61.8462,63.2100,60.6525
